In [1]:
#Lecture 21
from dwave.system.samplers import DWaveSampler

In [2]:
# 1) Implement simple QUBO with two qubits in the Chimera graph: E(q0,q1)=(q0-q1)^2
# Set Q for the minor-embedded problem QUBO
qubit_biases = {(0, 0): 1, (4, 4): 1}
coupler_strengths = {(0, 4): -2}
Q = dict(qubit_biases)
Q.update(coupler_strengths)

In [4]:
# Let's find out which quantum annealers are accessible to you. 
from dwave.cloud import Client
# Remove comment in the command below and add your token:
client = Client.from_config(token='DEV-3f171e3aed1d0db64a071c9147a304e3ed235e8d')
client.get_solvers()

[StructuredSolver(id='DW_2000Q_6'),
 StructuredSolver(id='Advantage_system1.1'),
 BQMSolver(id='hybrid_binary_quadratic_model_version2'),
 DQMSolver(id='hybrid_discrete_quadratic_model_version1')]

In [ ]:
# There are two quantum devices available for me. DW_2000Q_6 is a 2000 qubit quantum annealer with low noise. 
# Advantage_system1.1 is a 5000 quantum annealer with ~3X higher noise. 
# Let's use the 2000 qubit one:

In [5]:
response = DWaveSampler(id='DW_2000Q_6').sample_qubo(Q, num_reads=100)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{0: 0, 4: 0} Energy:  0.0 Occurrences:  56
{0: 1, 4: 1} Energy:  0.0 Occurrences:  44


In [6]:
response = DWaveSampler(id='Advantage_system1.1').sample_qubo(Q, num_reads=100)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{0: 0, 4: 0} Energy:  0.0 Occurrences:  40
{0: 1, 4: 1} Energy:  0.0 Occurrences:  60


In [7]:
# As you can see, most of the time the QPU outputs values with qubits in the same state (00 or 11).
# 2) Minor-embedding to implement 3-bit 3-SAT
# We will now use "chaining" to lock q0 and q5 in the same state. This will allow the use of 
# (q0, q4, q1, q5) to represent 3 qubits coupled to each other with the same coupling strengths. 
# Entering the biases and coupler strengths as described in the notes:
qubit_biases = {(0, 0): 0.33, (1, 1): -0.33, (4, 4): -0.33, (5, 5): 0.33}
coupler_strengths = {(0, 4): 0.66, (1, 4): 0.66, (1, 5): 0.66, (0, 5): -1}
Q = dict(qubit_biases)
Q.update(coupler_strengths)

In [8]:
response = DWaveSampler(id='DW_2000Q_6').sample_qubo(Q, num_reads=1000)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{0: 1, 1: 0, 4: 0, 5: 1} Energy:  -0.33999999999999997 Occurrences:  514
{0: 0, 1: 0, 4: 1, 5: 0} Energy:  -0.33000000000000007 Occurrences:  221
{0: 0, 1: 1, 4: 0, 5: 0} Energy:  -0.33000000000000007 Occurrences:  265


In [9]:
response = DWaveSampler(id='Advantage_system1.1').sample_qubo(Q, num_reads=1000)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{0: 1, 1: 0, 4: 0, 5: 1} Energy:  -0.33999999999999997 Occurrences:  493
{0: 0, 1: 0, 4: 1, 5: 0} Energy:  -0.33000000000000007 Occurrences:  184
{0: 0, 1: 1, 4: 0, 5: 0} Energy:  -0.33000000000000007 Occurrences:  322
{0: 0, 1: 0, 4: 0, 5: 0} Energy:  0.0 Occurrences:  1


In [10]:
# The trick worked out nicely! We got each of the 3 right answers. One of them 
# has slightly less energy and appeared about 1/2 of the time. 
# The other two have the same energy and appeared 1/4 of the time each. 
# 3) Using DWave's EmbeddingComposite function to do the minor-embedding for you
# Let's implement the 3-SAT example from class:
# Four bits z0 z1 z2 z3 z4
# Three clauses C(0,1,2), C(1,2,3), C(0,3,4)
from dwave.system.composites import EmbeddingComposite

In [11]:
# Set Q for the problem QUBO
linear = {('z0', 'z0'): -2, ('z1', 'z1'): -2, ('z2', 'z2'): -2, ('z3', 'z3'): -2, ('z4', 'z4'): -1}
quadratic = {('z0', 'z1'): 2, ('z0', 'z2'): 2, ('z1', 'z2'): 4, ('z1', 'z3'): 2, ('z2', 'z3'): 2, ('z0', 'z3'): 2, ('z0', 'z4'): 2, ('z3', 'z4'): 2}
Q = dict(linear)
Q.update(quadratic)

In [12]:
# Minor-embed and sample 1000 times on a default D-Wave system
response = EmbeddingComposite(DWaveSampler(id='DW_2000Q_6')).sample_qubo(Q, num_reads=1000)
for (sample, energy, num_occurrences, aux) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num_occurrences)

{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  617
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  317
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  3
{'z0': 0, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  2
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  12
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  3
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  5
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  16
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 1, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  6
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 1} Energy:  -1.0 Occurrences:  1
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -1.0 Occurrences:  2


In [ ]:
# Are these the correct solutions we agreed in class? :-)
# Note that only energy = -N_c (N_c is number of clauses) satisfies the 3-SAT problem ! 

In [13]:
# Let's try to use their new 5000 qubit device. It has more noise though. 
response_adv = EmbeddingComposite(DWaveSampler(id='Advantage_system1.1')).sample_qubo(Q, num_reads=1000)
for (sample, energy, num_occurrences, aux) in response_adv.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num_occurrences)

{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  517
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  451
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  1
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  2
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  5
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  7
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  3
{'z0': 1, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  5
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  7
{'z0': 0, 'z1': 1, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -1.0 Occurrences:  1
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 1, 'z4': 1} Energy:  -1.0 Occurrences:  1


In [ ]:
# The Advantage_system returned less high energy states! This is different than I expected given that it has higher noise. Maybe it's a rare event :-)